In [39]:
tr_data = [
['sunny','hot','high','weak','no'],
['sunny','hot','high','strong','no'],
['overcast','hot','high','weak','yes'],
['rain','mild','high','weak','yes'],
['rain','cool','normal','weak','yes'],
['rain','cool','normal','strong','no'],
['overcast','cool','normal','strong','yes'],
['sunny','mild','high','weak','no'],
['sunny','cool','normal','weak','yes'],
['rain','mild','normal','weak','yes'],
['sunny','mild','normal','strong','yes'],
['overcast','mild','high','strong','yes'],
['overcast','hot','normal','weak','yes'],
['rain','mild','high','strong','no'],
]
head = ["outlook","temperature","humidity","wind","decision"]

In [40]:
def unique_vals(Data,col):
    return set([row[col] for row in Data])

In [41]:
unique_vals(tr_data,0)

{'overcast', 'rain', 'sunny'}

In [42]:
def class_cnt(Data):
    cnt = {}
    for row in Data:
        label=row[-1]
        if label not in cnt:
            cnt[label] = 0
        cnt[label] += 1
    return cnt

In [43]:
class_cnt(tr_data)

{'no': 5, 'yes': 9}

In [44]:
class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    def match(self,example):
        val = example[self.column]
        return val == self.value
    def __repr__(self):
        return "IS %s %s %s ?" %(head[self.column]," == ",str(self.value))

In [45]:
def partition(Data,question):
    true_r,false_r = [],[]
    for row in Data:
        if(question.match(row)):
            true_r.append(row)
        else:
            false_r.append(row)
    return true_r,false_r

In [46]:
true_r,false_r = partition(tr_data,Question(0,"rainy"))
print(true_r)
print(false_r)
q = Question(0,"rainy")

[]
[['sunny', 'hot', 'high', 'weak', 'no'], ['sunny', 'hot', 'high', 'strong', 'no'], ['overcast', 'hot', 'high', 'weak', 'yes'], ['rain', 'mild', 'high', 'weak', 'yes'], ['rain', 'cool', 'normal', 'weak', 'yes'], ['rain', 'cool', 'normal', 'strong', 'no'], ['overcast', 'cool', 'normal', 'strong', 'yes'], ['sunny', 'mild', 'high', 'weak', 'no'], ['sunny', 'cool', 'normal', 'weak', 'yes'], ['rain', 'mild', 'normal', 'weak', 'yes'], ['sunny', 'mild', 'normal', 'strong', 'yes'], ['overcast', 'mild', 'high', 'strong', 'yes'], ['overcast', 'hot', 'normal', 'weak', 'yes'], ['rain', 'mild', 'high', 'strong', 'no']]


In [47]:
def gini(Data):
    counts = class_cnt(Data)
    impurity=1
    for lbl in counts:
        p_of_lbl = counts[lbl]/float(len(Data))
        impurity-=p_of_lbl**2
    return impurity

In [48]:
def info_gain(l,r,curr_uncertainity):
    p = float(len(l)/(len(l)+len(r)))
    return curr_uncertainity- p*gini(l)-(1-p)*gini(r)

In [49]:
def find_best_split(Data):
    best_gain=0
    best_ques= None
    curr_uncertainity = gini(Data)
    n_features = len(Data[0])-1
    for col in range(n_features):
        values = unique_vals(Data,col)
        for val in values:
            question = Question(col,val)
            true_r,false_r=partition(Data,question)
            if(len(true_r)==0 or len(false_r)==0):
                continue
            gain = info_gain(true_r,false_r,curr_uncertainity)
            if gain>=best_gain:
                best_gain,best_ques = gain,question
    return best_gain,best_ques

In [50]:
class Leaf: 
    def __init__(self,Data):
        self.predictions = class_cnt(Data)

In [51]:
class Decision_Node:
        def __init__(self,question,true_branch,false_branch):
            self.question=question
            self.true_branch=true_branch
            self.false_branch=false_branch
            #print(self.question)

In [52]:
def build_tree(Data,i=0):
    gain,question = find_best_split(Data)
    if gain==0:
        return Leaf(Data)
    true_r,false_r = partition(Data,question)
    true_branch=build_tree(true_r,i)
    false_branch=build_tree(false_r,i)
    return Decision_Node(question,true_branch,false_branch)

In [53]:
def prnt_Tree(node,spacing=""):
    if isinstance(node,Leaf):
        print(spacing+"Predict",node.predictions)
        return
    print(spacing+str(node.question))
    print(spacing+"--> True:")
    prnt_Tree(node.true_branch,spacing+"\t")
    print(spacing+"--> False:")
    prnt_Tree(node.false_branch,spacing+"\t")

In [54]:
my_tree = build_tree(tr_data,0)
prnt_Tree(my_tree)

IS outlook  ==  overcast ?
--> True:
	Predict {'yes': 4}
--> False:
	IS humidity  ==  high ?
	--> True:
		IS outlook  ==  sunny ?
		--> True:
			Predict {'no': 3}
		--> False:
			IS wind  ==  strong ?
			--> True:
				Predict {'no': 1}
			--> False:
				Predict {'yes': 1}
	--> False:
		IS wind  ==  strong ?
		--> True:
			IS temperature  ==  mild ?
			--> True:
				Predict {'yes': 1}
			--> False:
				Predict {'no': 1}
		--> False:
			Predict {'yes': 3}


In [55]:
def prnt_leaf(cnt):
    total = sum(cnt.values())*1.0
    probs = {}
    for lbl in cnt.keys():
        probs[lbl]=str(int(cnt[lbl]/total*100))+"%"
    return probs

In [56]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [67]:
test_data = [["overcast","cool","high","strong","no"],["sunny","cool","normal","strong","yes"]]

In [68]:
for row in test_data:
    print("Actual: %s. Prediction : %s" %(row[-1],prnt_leaf(classify(row,my_tree))))

Actual: no. Prediction : {'yes': '100%'}
Actual: yes. Prediction : {'no': '100%'}
